#Designing helical bundles
<font color='grey' size='1.5'> Created by Parisa Hosseinzadeh for *Protein Engineering and Design*, Winter 2022


In this exercise, you will get familiar with one of the most successful protein design methods: helical bundle design.

Make sure you download this code and add it to your personal google drive.

You can read more about the application of this method in the following papers:
1. De novo design of covalently constrained mesosize protein scaffolds with unique tertiary structures ([link](https://www.ncbi.nlm.nih.gov/pubmed/28973862))
2. Accurate computational design of multipass transmembrane proteins ([link]( https://www.ncbi.nlm.nih.gov/pubmed/29496880))
3. Programmable design of orthogonal protein heterodimers ([link](https://www.ncbi.nlm.nih.gov/pubmed/30568301))
4. De novo design of bioactive protein switches ([link](https://www.ncbi.nlm.nih.gov/pubmed/31341284))

<font color='purple' size='2'> The pyrosetta code is originally written by Jared Adolf-Bryfogle and the underlying C++ code was implemented by Vikram Khipple Mulligan. For more information abour the code you can look at Rosetta Commons webpage on [MakeBundleMover](https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/MakeBundleMover) and [BundleGridSamplerMover](https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/BundleGridSamplerMover).

##Step 1. Preparing for the run

The following steps walk you through initial steps to set up everything you need for the run.

### Preparing Pyrosetta on your computer



1.   Create a folder in your google drive and name it **PyRosetta**
2.   Download `pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl` from your course files and put it in the PyRosetta folder.



In [1]:
#@title Mounting Google Drive
#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. 


google_drive_mount_point = '/content/google_drive'

import os, sys, time

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount(google_drive_mount_point)

if not os.getenv("DEBUG"):
    google_drive = google_drive_mount_point + '/My Drive' 

Drive already mounted at /content/google_drive; to attempt to forcibly remount, call drive.mount("/content/google_drive", force_remount=True).


In [2]:
#@title Going to a directory of interest
#@markdown Running this cell will change the directory
#@markdown to one in your google drive.

path = '/content/google_drive/MyDrive'#@param {type:"string"}

!cd $path

In [3]:
#@title Installing PyRosetta
#@markdown Download pyrosetta. 
#@markdown This cell will take a few min to run.

%%time
if not os.getenv("DEBUG"):
    # installing PyRosetta
    if sys.version_info.major != 3 or sys.version_info.minor != 7:
        print('Need Python-3.7 to run!')
        sys.exit(1)

    # upload PyRosetta Linux WHEEL package into your google drive and put it into /PyRosetta dir
    # or alternatively you can download PyRosetta directly from GrayLab web site (but this might take some time!)
    #!mkdir $notebook_path/PyRosetta
    #!cd $notebook_path/PyRosetta && wget --user USERNAME --password PASSWORD https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python37.ubuntu.wheel/latest.html   

    pyrosetta_distr_path = google_drive + '/PyRosetta' 
    
    # finding path to wheel package, if multiple packages is found take first one
    # replace this with `wheel_path = pyrosetta_distr_path + /<wheel-file-name>.whl` if you want to use particular whl file
    wheel_path = pyrosetta_distr_path + '/' + [ f for f in os.listdir(pyrosetta_distr_path) if f.endswith('.whl')][0]
    
    print(f'Using PyRosetta wheel package: {wheel_path}')

    !pip3 install '{wheel_path}' 

Using PyRosetta wheel package: /content/google_drive/My Drive/PyRosetta/pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl
Processing ./google_drive/My Drive/PyRosetta/pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl
pyrosetta is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
CPU times: user 51.2 ms, sys: 13.4 ms, total: 64.6 ms
Wall time: 5.25 s


In [4]:
#@title Importing necessary modules

#importing modules needed from pyrosetta
from pyrosetta import init
from pyrosetta.rosetta.core.pose import Pose
import pyrosetta.rosetta.protocols.rosetta_scripts as rosetta_scripts
from pyrosetta.rosetta.protocols.helical_bundle import *

#required for visualization
!pip install -q py3Dmol
import py3Dmol

#import other modules
import math

import logging
logging.basicConfig(level=logging.INFO)

##Step 2. Testing parameters

In this step, we will scan different parameters to find the best range for generating our helical bundles.

In [5]:
#@title Initializing pyrosetta

init("-mute all")

INFO:pyrosetta.rosetta:Found rosetta database at: /usr/local/lib/python3.7/dist-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [6]:
#@title 2.1. Sampling r0
#@markdown By running this cell, we will sample
#@markdown different values for r0 and check their
#@markdown effect on the output.

length = 20 #@param {type:"number"}

#@markdown - `length` is the size of helices to sample.

r0_min = 1 #@param {type:"number"}
r0_max = 10 #@param {type:"number"}
r0_samples =  5#@param {type:"number"}

#@markdown - `r0_min` Minimum r0 value to sample.
#@markdown -  `mutation_rate` Maximum r0 value to sample.
#@markdown -  `library_size` Number of samples. Keep it at **<10** for class.

#creating the code to run for sampling
sampler_string = f'''
<SCOREFXNS>
  <ScoreFunction name="sfxn1" weights="ref2015"/>
</SCOREFXNS>
<MOVERS>
  <BundleGridSampler name="bgs1"
                     helix_length="{length}"
                     scorefxn="sfxn1"
                     r0_min="{r0_min}"
                     r0_max="{r0_max}"
                     r0_samples="{r0_samples}"
                     omega0="0.05"
                     delta_omega0="0"
                     delta_omega1="0"
                     delta_t="0"
                     dump_pdbs="1">
  <Helix/>
  <Helix delta_omega0="3.14" r0_copies_helix="1"/>
</BundleGridSampler>
</MOVERS>'''

#generating an empty pose
pose = Pose()
#running
rosetta_scripts.XmlObjects.create_from_string(
    sampler_string).get_mover("bgs1").apply(pose)

In [7]:
#@title Visualization of the results
#@markdown The results are saved as `bgs_out_000*.pdb`.
#@markdown you can download and look at them in pymol,
#@markdown or visualize them here.

pdb_name = 'bgs_out_00005.pdb' #@param {type:"string"}

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open(pdb_name,'r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
#@title 2.2. Sampling omega

length = 20 #@param {type:"number"}

#@markdown - `length` is the size of helices to sample.

o_min =  0#@param {type:"number"}
o_max =  32#@param {type:"number"}
o_steps =  2#@param {type:"number"}

#@markdown - `o_min` Minimum omega value (degrees) to sample.
#@markdown -  `o_max` Maximum o value to sample.
#@markdown -  `steps to sample` difference between each step (2 means 0 --> 2 --> 4 --> 6 --> ...)

for i in range(o_min, o_max, o_steps):

    i_rad = math.radians(i)
    pose = Pose()
    print(i_rad)

    sampler_string = f'''
    <SCOREFXNS>
        <ScoreFunction name="sfxn1" weights="ref2015"/>
    </SCOREFXNS>
    <MOVERS>
        <MakeBundle name="bgs1"
                     helix_length="{length}"
                     r0="5"
                     omega0="{i_rad}"
                     delta_omega0="0"
                     delta_omega1="0"
                     delta_t="0">
        <Helix/>
        <Helix delta_omega0="3.14" r0_copies_helix="1"/>
        
        </MakeBundle>
    </MOVERS>'''
    rosetta_scripts.XmlObjects.create_from_string(sampler_string).get_mover("bgs1").apply(pose)
    pose.dump_pdb('omega0_'+str(i)+'.pdb')

0.0
0.03490658503988659
0.06981317007977318
0.10471975511965978
0.13962634015954636
0.17453292519943295
0.20943951023931956
0.24434609527920614
0.2792526803190927
0.3141592653589793
0.3490658503988659
0.3839724354387525
0.4188790204786391
0.4537856055185257
0.4886921905584123
0.5235987755982988


In [9]:
#@title Visualization of the results
#@markdown The results are saved as `omega0_*.pdb`
#@markdown where * is the value of degree.
#@markdown you can download and look at them in pymol,
#@markdown or visualize them here.

pdb_name = 'omega0_4.pdb' #@param {type:"string"}

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open(pdb_name,'r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

##Step 3. Create your own bundle manually

Based on the parameters we described, find the best omega and r0 and create your very own helical bundle!

In [10]:
#@title bundle generation

length = 20 #@param {type:"number"}

#@markdown - `length` is the size of helices to sample.

r_value= 5 #@param {type:"number"}
omega_value= 12 #@param {type:"number"}

#generate a pose
pose =Pose()

#create an instance of make bundle
mb = MakeBundle()

#add required number of helices
mb.add_helix()
mb.add_helix()
mb.helix(2).calculator_op().real_parameter(BPC_delta_omega0).set_value(3.14)

for i in range(1, 3):
    mb.helix(i).set_helix_length(length)
    mb.helix(i).calculator_op().real_parameter(BPC_r0).set_value(r_value)
    mb.helix(i).calculator_op().real_parameter(BPC_omega0).set_value(math.radians(omega_value))

mb.apply(pose)
pose.dump_pdb("manual_bundle.pdb")

True

In [11]:
#@title Visualization of the results
#@markdown The results is saved as `manual_bundle.pdb`.
#@markdown you can download and look at them in pymol,
#@markdown or visualize them here.

pdb_name = 'manual_bundle.pdb' #@param {type:"string"}

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open(pdb_name,'r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol